# TSA Claims Analysis

## by Justin Sierchio

In this analysis, we will be looking at claims made against the US Transportation Security Administration (TSA) from 2002 to 2015. Ideally, we would like to be able to answer the following questions:

<ul>
    <li>Which airports have the most complaints?</li>
    <li>What are the most likely claims made?</li>
    <li>Are there certain times of the year where more incidents are likely to occur?</li>
</ul>

This data is in .csv file format and is from Kaggle at: https://www.kaggle.com/perrychu/tsa-claims-classification-part-1. More information related to the dataset can be found at: https://www.kaggle.com/terminal-security-agency/tsa-claims-database.

Special thanks to 'perrychu' at Kaggle for their data cleaning procedure.

## Notebook Initialization

In [1]:
# Import Relevant Libraries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import re

print('Initial libraries loaded into workspace!')

Initial libraries loaded into workspace!


In [2]:
# Upload Datasets for Study
df_TSA = pd.read_csv("tsa_claims_clean.csv");

print('Datasets uploaded!');

Datasets uploaded!


C:\Users\jmsie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Display 1st 5 rows from TSA Claims dataset
df_TSA.head()

,Claim_Number,Airline_Name,Claim_Type,Claim_Site,Item,Status,Date_Received,Incident_Date,Airport_Code_Group,Airport_Name_Group,Claim_Value,Close_Value
0,0909802M,ContinentalAirlines,Property Damage,Checkpoint,Other,Approved,2002-01-04,2002-12-12,EWR,Newark International Airport,350.00,350.00
1,0202417M,-,Property Damage,Checked Baggage,Luggage (all types including footlockers),Settled,2002-02-02,2004-01-16,SEA,Seattle-Tacoma International,100.00,50.00
2,0202445M,AmericanAirlines,Property Damage,Checked Baggage,Cell Phones,Settled,2002-02-04,2003-11-26,STL,Lambert St. Louis International,278.88,227.92
3,0909816M,AmericanAirlines,Property Damage,Checkpoint,Luggage (all types including footlockers),Approved,2002-02-07,2003-01-06,MIA,Miami International Airport,50.00,50.00
4,2005032379513,DeltaAirLines,Property Damage,Checkpoint,Baby - Strollers; car seats; playpen; etc.,Approved,2002-02-18,2005-02-05,MCO,Orlando International Airport,84.79,84.79


Let's describe what each of the columns in this dataset mean.

<ul>
    <li>Claim_Number: Self-Explanatory</li>
    <li>Airline_Name: Self-Explanatory</li>
    <li>Claim_Type: Self-Explanatory</li>
    <li>Claim_Site: Where did the incident occur?</li>
    <li>Item: What kind of item was the incident involving?</li>
    <li>Status: What happened with the claim?</li>
    <li>Date_Received: When was the claim received?</li>
    <li>Incident_Date: When did the claim incident occur?</li>
    <li>Airport_Code_Group: Code name for the airport</li>
    <li>Airport_Name_Group: Name of the airport?</li>
    <li>Claim_Value: Value of the claim?</li>
    <li>Close_Value: Value for settling the claim.</li>
</ul>

## Data Information

Let's show the basic data statistics. 

In [8]:
df_TSA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145012 entries, 0 to 145011
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Claim_Number        145012 non-null  object 
 1   Airline_Name        145012 non-null  object 
 2   Claim_Type          145012 non-null  object 
 3   Claim_Site          145012 non-null  object 
 4   Item                145012 non-null  object 
 5   Status              145012 non-null  object 
 6   Date_Received       145012 non-null  object 
 7   Incident_Date       145012 non-null  object 
 8   Airport_Code_Group  145012 non-null  object 
 9   Airport_Name_Group  145012 non-null  object 
 10  Claim_Value         145012 non-null  float64
 11  Close_Value         128542 non-null  float64
dtypes: float64(2), object(10)
memory usage: 13.3+ MB


So we have 145k+ incidents to work with. Let's get started!

## Exploratory Data Analysis

To begin our analysis, let's look at the Top 20 airports for TSA claims. Since we've removed some data, we'll show the results in percentages rather than raw numbers.

In [4]:
# List the Top 20 Airports for TSA Complaints
print('Here are the Top 20 Airports with TSA Claims 2002-2015:\n')
print('Airport Name                         # TSA Claims (% of Total)\n')
df_TSA_TypePerc = 100. * df_TSA['Airport_Name_Group'].value_counts() / len(df_TSA);
print(df_TSA_TypePerc.head(20))

Here are the Top 20 Airports with TSA Claims 2002-2015:

Airport Name                         # TSA Claims (% of Total)

Other                                               7.152512
Los Angeles International Airport                   4.909249
John F. Kennedy International                       3.965879
Chicago O'Hare International Airport                3.635561
Newark International Airport                        3.599012
Miami International Airport                         3.070091
Orlando International Airport                       3.035611
Seattle-Tacoma International                        2.841144
Hartsfield-Jackson Atlanta International Airport    2.726671
Phoenix Sky Harbor International                    2.591510
-                                                   2.568063
McCarran International                              2.446004
Philadelphia International Airport                  2.373597
Dallas-Fort Worth International Airport             2.238435
Ft. Lauderdale-Hollywood 

As we can see from our cleaned, complete dataset, LAX (Los Angeles International Airport) had the most TSA claims, followed by JFK (New York City - John F. Kennedy International Airport) and ORD (Chicago O'Hare International Airport).

Let's take a look at the frequency of types of claims next.

In [5]:
# List the types of TSA Claims
print('Here are the Top Claim Types with the TSA 2002-2015:\n')
print('Claim Type           # TSA Claims (% of Total)\n')
df_TSA_ClaimTypePerc = 100. * df_TSA['Claim_Type'].value_counts() / len(df_TSA);
print(df_TSA_ClaimTypePerc)

Here are the Top Claim Types with the TSA 2002-2015:

Claim Type           # TSA Claims (% of Total)

Passenger Property Loss    56.435329
Property Damage            38.712658
-                           3.642457
Personal Injury             0.475823
Passenger Theft             0.294458
Employee Loss (MPCECA)      0.288942
Motor Vehicle               0.147574
Complaint                   0.002069
Bus Terminal                0.000690
Name: Claim_Type, dtype: float64


According to our analysis, the vast majority of claims involve passenger property loss and property damage.

Now let's look at the item list and airline names, respectively.

In [6]:
# List the items for TSA Claims
print('Here are the Top Items with the TSA 2002-2015:\n')
print('Item Type                                            # TSA Claims (% of Total)\n')
df_TSA_ItemTypePerc = 100. * df_TSA['Item'].value_counts() / len(df_TSA);
print(df_TSA_ItemTypePerc.head(10))

Here are the Top Items with the TSA 2002-2015:

Item Type                                            # TSA Claims (% of Total)

Other                                                           19.945936
Luggage (all types including footlockers)                       11.868673
Clothing - Shoes; belts; accessories; etc.                       9.191653
Locks                                                            6.131217
Jewelry - Fine                                                   5.885030
Cameras - Digital                                                5.743663
Computer - Laptop                                                5.316801
-                                                                2.561857
Eyeglasses - (including contact lenses)                          2.473588
Cosmetics - Perfume; toilet articles; medicines; soaps; etc.     2.217058
Name: Item, dtype: float64


As one might expect, luggage was the most frequent item at about 15%. Also perhaps not surprising on this top 10 list were digital cameras, fine jewelry, laptop computers, cosmetics, currency and cell phones. A possible hypothesis is that certain entities are committing theft at airports.

In [7]:
# List the Top Airlines for TSA Complaints
print('Here are the Top 20 Airlines with TSA Claims 2002-2015:\n')
print('Airline Name         # TSA Claims (% of Total)\n')
df_TSA_AirlineTypePerc = 100. * df_TSA['Airline_Name'].value_counts() / len(df_TSA);
print(df_TSA_AirlineTypePerc.head(20))

Here are the Top 20 Airlines with TSA Claims 2002-2015:

Airline Name         # TSA Claims (% of Total)

-                      19.801120
DeltaAirLines          11.992111
AmericanAirlines       11.854881
UAL                     9.355088
SouthwestAirlines       8.993049
ContinentalAirlines     6.628417
USAir                   6.536011
NorthwestAirlines       5.911235
JetBlue                 3.303865
AlaskaAirlines          3.296279
AirTranAirlines         2.332910
AmericaWest             1.162662
FrontierAirlines        1.035087
SpiritAirlines          0.826139
HawaiianAirlines        0.619949
BritishAirways          0.530301
AllegiantAir            0.428930
AirCanada               0.428240
Lufthansa               0.378589
VirginAtlantic          0.328938
Name: Airline_Name, dtype: float64


As our analysis shows from our cleaned dataset, American Airlines had a higher % of claims than any other airline with near 15% of the overall total. Rounding out the top 5 were Delta, United, Southwest and Continental. [Note this set was collected before the AA-USAir merger].

Now let's work on answering the question regarding time of year or holidays and any relation to claims.